<a href="https://colab.research.google.com/github/samKhallaghi/02456-deep-learning-with-PyTorch/blob/master/graded_assignment5_rrf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Applying Deep Learning to Earth Observation - Assignment 5**

## Instructions

Work through the assignment 5 notebook, and use this notebook to provide your answers. 

To submit the assignment, you will need to use GitHub and the existing private repository you already created called `adleoxyz` (xyz is your initials)

Once you have completed the assignment:
- Commit your notebook from colab to your private GitHub repo
- The notebook should be named assignment4_xyz.ipynb, with xyz again replaced by your initials.
- There are 50 points in this assignment, with an additional 5 extra credit. 

# **Theoretical questions**

## **Answer to Q1** 

## **Answer to Q2** 

# **Continue and complete our pipeline implementation**

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%%capture
!pip install rasterio

In [ ]:
import os
import math
from pathlib import Path
from datetime import datetime, timedelta
import tqdm
import pandas as pd
import numpy as np
import cv2
import random
import rasterio

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.tensorboard import SummaryWriter

from IPython.core.debugger import set_trace
from IPython.display import Image

## **Utillity functions**

In [ ]:
def load_data(data_path, is_label=False, apply_normalization=False, dtype=np.float32, verbose=False):
    r"""
    Open data using gdal, read it as an array and normalize it.
    Arguments:
            data_path (string) -- Full path including filename of the data source we wish to load.
            is_label (binary) -- If True then the layer is a ground truth (category index) and if
                                set to False the layer is a reflectance band.
            apply_normalization (binary) -- If true min/max normalization will be applied on each band.
            dtype (np.dtype) -- Data type of the output image chips.
            verbose (binary) -- if set to true, print a screen statement on the loaded band.
    Returns:
            image -- Returns the loaded image as a 32-bit float numpy ndarray with shape (height, width, num_bands).
    """

    # Inform user of the file names being loaded from the Dataset.
    if verbose:
        print('loading file:{}'.format(data_path))

    # open dataset using rasterio library.
    with rasterio.open(data_path, "r") as src:

        if is_label:
            if src.count != 1:
                raise ValueError("Expected Label to have exactly one channel.")
            img = src.read(1)

        else:
            if apply_normalization:
                img = min_max_normalize_image(src.read())
                img = img.astype(dtype)
            else:
                img = src.read()
                img = img.astype(dtype)

    return img

#########################

def get_meta_from_bounds(image, overlap=None):
    with rasterio.open(image, "r") as src:
        meta = src.meta
    
    if overlap:
        dst_width = src.width - 2 * overlap
        dst_height = src.height - 2 * overlap
        window = Window(overlap, overlap, dst_width, dst_height)
        win_transform = src.window_transform(window)
        meta.update({
            'width': dst_width,
            'height': dst_height,
            'transform': win_transform,
            'count': 1,
            'dtype': 'int8'
        })

    return meta

#########################

def load_params(params_dir, model, freeze_params=None):
    """Load a set of parameters into a PyTorch model.
    Args:
        params_dir (str): Path to the ".pth" or ".pt" file containing the input parameters.
        model (pytorch nn object): Initialized model.
        freeze_params (list[int] or None, optional): List of parameter indices to freeze (i.e., set to
            require no gradients). Default is None, which means that all parameters are trainable.

    Returns:
        None

    Raises:
        FileNotFoundError: If the specified file does not exist.

    Example:
        # Load parameters from file and freeze the first layer
        load_params("params.pth", freeze_params=[0])
    """
    # Load input parameters from file
    input_params = torch.load(params_dir)

    # Filter out parameters that are not in the model    
    model_dict = model.state_dict()
    if "module" in list(input_params.keys())[0]:
        input_params_filter = {k[7:]: v.cpu() for k, v in input_params.items() if k[7:] in model_dict}
    else:
        input_params_filter = {k: v.cpu() for k, v in input_params.items() if k in model_dict}

    # Update the model parameters with the input parameters    
    model_dict.update(input_params_filter)
    model.load_state_dict(model_dict)

    # free some layers
    if freeze_params is not None:
        for i, p in enumerate(model.parameters()):
            if i in freeze_params:
                p.requires_grad = False
    
    return model

## **Pre-processing steps**

In [ ]:
#@title Code for input normalization

def min_max_normalize_image(image, dtype=np.float32):
    """
    image_path(str) : Absolute path to the image patch.
    dtype (numpy datatype) : data type of the normalized image default is "np.float32".
    """

    # Calculate the minimum and maximum values for each band
    min_values = np.nanmin(image, axis=(1, 2))[:, np.newaxis, np.newaxis]
    max_values = np.nanmax(image, axis=(1, 2))[:, np.newaxis, np.newaxis]

    # Normalize the image data to the range [0, 1]
    normalized_img = (image - min_values) / (max_values - min_values)

    # Return the normalized image data
    return normalized_img

In [ ]:
#@title Code for image augmentation techniques

def flip_image_and_label(image, label, flip_type):
    """
    Applies horizontal or vertical flip augmentation to an image patch and label

    Args:
        image (numpy array) : The input image patch as a numpy array.
        label (numpy array) : The corresponding label as a numpy array.
        flip_type (string) : Based on the direction of flip. Can be either 
            'hflip' or 'vflip'.

    Returns:
        A tuple containing the flipped image patch and label as numpy arrays.
    """
    if flip_type == 'hflip':
        # Apply horizontal flip augmentation to the image patch
        flipped_image = cv2.flip(image, 1)
        
        # Apply horizontal flip augmentation to the label
        flipped_label = cv2.flip(label, 1)
        
    elif flip_type == 'vflip':
        # Apply vertical flip augmentation to the image patch
        flipped_image = cv2.flip(image, 0)
        
        # Apply vertical flip augmentation to the label
        flipped_label = cv2.flip(label, 0)
        
    else:
        raise ValueError("Flip direction must be 'horizontal' or 'vertical'.")
        
    # Return the flipped image patch and label as a tuple
    return flipped_image.copy(), flipped_label.copy()


def rotate_image_and_label(image, label, angle):
    """
    Applies rotation augmentation to an image patch and label.

    Args:
        image (numpy array) : The input image patch as a numpy array.
        label (numpy array) : The corresponding label as a numpy array.
        angle (lost of floats) : If the list has exactly two elements they will
            be considered the lower and upper bounds for the rotation angle 
            (in degrees) respectively. If number of elements are bigger than 2, 
            then one value is chosen randomly as the roatation angle.

    Returns:
        A tuple containing the rotated image patch and label as numpy arrays.
    """
    if isinstance(angle, tuple) or isinstance(angle, list):
        if len(angle) == 2:
            rotation_degree = random.uniform(angle[0], angle[1])
        elif len(angle) > 2:
            rotation_degree = random.choice(angle)
        else:
            raise ValueError("Parameter degree needs at least two elements.")
    else:
        raise ValueError(
            "Rotation bound param for augmentation must be a tuple or list."
        )
    
    # Define the center of the image patch
    center = tuple(np.array(label.shape)/2.0)

    # Define the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center, rotation_degree, 1.0)

    # Apply rotation augmentation to the image patch
    rotated_image = cv2.warpAffine(image, rotation_matrix, image.shape[:2], 
                                   flags=cv2.INTER_LINEAR)

    # Apply rotation augmentation to the label
    rotated_label = cv2.warpAffine(label, rotation_matrix, label.shape[:2], 
                                   flags=cv2.INTER_NEAREST)

    # Return the rotated image patch and label as a tuple
    return rotated_image.copy(), np.rint(rotated_label.copy())

In [ ]:
#@title Code to get the index of center of each smaller chip

def patch_center_index(cropping_ref, patch_size, overlap, usage, 
                       positive_class_threshold=None, verbose=True):
    r"""
    Generate index to divide the scene into small chips.
    Each index marks the location of corresponding chip center.
    Arguments:
        cropping_ref (list) : Reference raster layers, to be used to generate 
            the index. In our case, it is study area binary mask and label mask.
        patch_size (int) : Size of each clipped patches.
        overlap (int) : amount of overlap between the extracted chips.
        usage (str) : Either 'train', 'val'. Chipping strategy is different for 
            different usage.
        positive_class_threshold (float) : A real value as a threshold for the 
            proportion of positive class to the total areal of the chip. Used to 
            decide if the chip should be considered as a positive chip in the 
            sampling process.
    verbose (binary) : If set to True prints on screen the detailed list of 
            center coordinates of the sampled chips.
    Returns:
        proportional_patch_index : A list of index recording the center of 
        patches to extract from the input
    """

    assert usage in ["train", "validation", "inference"]

    if usage == "inference":
        mask = cropping_ref
    else:
        mask, label = cropping_ref

    half_size = patch_size // 2
    step_size = patch_size - 2 * overlap

    proportional_patch_index = []
    non_proportional_patch_index = []
    neg_patch_index = []

    # Get the index of all the non-zero elements in the mask.
    x = np.argwhere(mask)

    # First col of x shows the row indices (height) of the mask layer 
    # (iterate over the y axis or latitude).
    x_min = min(x[:, 0]) + half_size
    x_max = max(x[:, 0]) - half_size
    # Second col of x shows the column indices (width) of the mask layer 
    # (iterate over the x axis or longitude).
    y_min = min(x[:, 1]) + half_size
    y_max = max(x[:, 1]) - half_size

    # Generate index for the center of each patch considering the proportion of 
    # each category falling into each patch.
    for j in range(y_min, y_max + 1, step_size):

        for i in range(x_min, x_max + 1, step_size):

            # Split the mask and label layers into patches based on the index of 
            # the center of the patch
            mask_ref = mask[i - half_size: i + half_size, 
                            j - half_size: j + half_size]
            if usage != "inference":
                label_ref = label[i - half_size: i + half_size, 
                                  j - half_size: j + half_size]

            if (usage == "train") and mask_ref.all():

                if label_ref.any() != 0:
                    pond_ratio = np.sum(label_ref == 1) / label_ref.size
                    if pond_ratio >= positive_class_threshold:
                        proportional_patch_index.append([i, j])
                else:
                    neg_patch_index.append([i, j])

            if (usage == "validation") and (label_ref.any() != 0) and mask_ref.all():
                non_proportional_patch_index.append([i, j])
            
            if (usage == "inference") and (mask_ref.any() != 0):
                non_proportional_patch_index.append([i, j])

    if usage == "train":

        num_negative_samples = min(
            math.ceil(0.2 * len(proportional_patch_index)), 15
        )
        neg_samples = random.sample(neg_patch_index, num_negative_samples)

        proportional_patch_index.extend(neg_samples)

    # For test set use the indices generated from mask without 
    # considering the class proportions.
    if usage in ["validation", "inference"]:
        proportional_patch_index = non_proportional_patch_index

    if verbose:
        print("Number of patches:", len(proportional_patch_index))
        print("Patched from:\n{}".format(proportional_patch_index))

    return proportional_patch_index

We need to modify our custom dataset to handle the "inference" phase (also called "prediction" or "test").

The process is very similar to the custom dataset you have used in assignment 3. The prediction scenes are organized in a "csv" file. Pixel values are atmospherically corrected to ground reflectance (Values are between 0 and 1) if you want you can standardize it or leave it as is.



In [ ]:
#@title Code for Custom dataset -- MUST get modified

class AquacultureData(Dataset):
    def __init__(self, src_dir, usage, dataset_name=None, apply_normalization=False, 
                 transform=None, csv_name=None, patch_size=None, overlap=None, catalog_index=None):
        r"""
        src_dir (str or path): Root of resource directory.
        dataset_name (str): Name of the training/validation dataset containing 
                              structured folders for image, label
        usage (str): Either 'train' or 'validation'.
        transform (list): Each element is string name of the transformation to be used.
        """
        self.src_dir = src_dir
        self.dataset_name = dataset_name
        self.csv_name = csv_name
        self.apply_normalization = apply_normalization
        self.transform = transform
        self.patch_size = patch_size
        self.overlap = overlap
        
        self.usage = usage
        assert self.usage in ["train", "validation", "inference"], "Usage is not recognized."

        if self.usage in ["train", "validation"]:
            assert self.dataset_name is not None
            img_dir = Path(src_dir) / self.dataset_name / self.usage / "bands"
            img_fnames = [Path(dirpath) / f for (dirpath, dirnames, filenames) in os.walk(img_dir) for f in 
                          filenames if f.endswith(".tif")]
            img_fnames.sort()
        
            lbl_dir = Path(src_dir) / self.dataset_name / self.usage / "labels"
            lbl_fnames = [Path(dirpath) / f for (dirpath, dirnames, filenames) in os.walk(lbl_dir) for f in filenames if f.endswith(".tif")]
            lbl_fnames.sort()

            self.img_chips = []
            self.lbl_chips = []
        
            for img_path, lbl_path in tqdm.tqdm(zip(img_fnames, lbl_fnames), total=len(img_fnames)):
                img_chip = load_data(img_path, is_label=False, apply_normalization=self.apply_normalization)
                img_chip = img_chip.transpose((1, 2, 0))

                lbl_chip = load_data(lbl_path, is_label=True)

                self.img_chips.append(img_chip)
                self.lbl_chips.append(lbl_chip)
            
            print('--------------{} patches cropped--------------'.format(len(self.img_chips)))
        
        # This part handles prediction dataset
        else:
            assert self.csv_name is not None
            
            ##### Add your code to read the "csv" file. (Expected 1 line)
            catalog = pd.read_csv(os.path.join(self.src_dir, self.csv_name))
            ##### use "iloc" and "catalog_index" to grab one line of catalog. (Expected 1 line)
            self.catalog = catalog.iloc[catalog_index]

            self.tile = (self.catalog["wrs_path"], self.catalog["wrs_row"])

            img_path_ls = [self.catalog["img_dir"]]
            mask_path_ls = [self.catalog["mask_dir"]]
            
            self.meta = get_meta_from_bounds(Path(src_dir) / img_path_ls[0])

            half_size = self.patch_size // 2

            self.img_chips = []
            self.coor = []

            for img_path, mask_path in zip(img_path_ls, mask_path_ls):
                
                ###### Add your code to load the image and assign it to a variable called "img".
                ###### Use the "load_data" function, provided in the utility function. (Expected 1 line)
                img = load_data(img_path)

                img = img.transpose((1, 2, 0))
                
                ##### Load your mask again using "load_data" function. (Expected 1 line)
                mask = load_data(mask_path)
                #mask = load_data(os.path.join(self.src_dir, mask_path), is_label=True)
                
                crop_ref = mask

                index = patch_center_index(crop_ref, self.patch_size, self.overlap, self.usage)

                for i in range(len(index)):
                    x = index[i][0]
                    y = index[i][1]

                    self.img_chips.append(img[x - half_size: x + half_size, y - half_size: y + half_size, :])
                    self.coor.append([x, y])

            
        
            print('--------------{} patches cropped--------------'.format(len(self.img_chips)))
        

    def __getitem__(self, index):

        if self.usage in ["train", "validation"]:
            image_chip = self.img_chips[index]
            label_chip = self.lbl_chips[index]

            if self.usage == "train" and self.transform:
                trans_flip_ls = [m for m in self.transform if "flip" in m]
                if random.randint(0, 1) and len(trans_flip_ls) > 1:
                    trans_flip = random.sample(trans_flip_ls, 1)[0]
                    image_chip, label_chip = flip_image_and_label(image_chip, label_chip, trans_flip)
            
                if random.randint(0, 1) and "rotate" in self.transform:
                    img_chip, lbl_chip = rotate_image_and_label(image_chip, label_chip, angle=[0,90])

            # Convert numpy arrays to torch tensors.
            # Image chips should be: CHW if not transpose to correct order of dimensions.
            image_tensor = torch.from_numpy(image_chip.transpose((2, 0, 1))).float()
            label_tensor = torch.from_numpy(np.ascontiguousarray(label_chip)).long()

            return image_tensor, label_tensor
        else:
            coor = self.coor[index]
            img_chip = self.img_chips[index]
            image_tensor = torch.from_numpy(img_chip.transpose((2, 0, 1))).float()

            return image_tensor, coor


    def __len__(self):

        return len(self.img_chips)

**<font color='red'> 23/25 points on CA1</font>**

**<font color='red'> Check line 80 that I added.</font>**

## **Model Architecture**

In [ ]:
#@title Add the Unet model you have designed from assignment 3 or use the existing one.

class ConvBlock(nn.Module):
    r"""This module creates a user-defined number of conv+BN+ReLU layers.
    Args:
        in_channels (int)-- number of input features.
        out_channels (int) -- number of output features.
        kernel_size (int) -- Size of convolution kernel.
        stride (int) -- decides how jumpy kernel moves along the spatial 
            dimensions.
        padding (int) -- how much the input should be padded on the borders with 
            zero.
        dilation (int) -- dilation ratio for enlarging the receptive field.
        num_conv_layers (int) -- Number of conv+BN+ReLU layers in the block.
        drop_rate (float) -- dropout rate at the end of the block.
    """

    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1,
                 padding=1, dilation=1, num_conv_layers=2, drop_rate=0):
        super(ConvBlock, self).__init__()

        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,
                            stride=stride, padding=padding, dilation=dilation, 
                            bias=False),
                  nn.BatchNorm2d(out_channels),
                  nn.ReLU(inplace=True), ]

        if num_conv_layers > 1:
            if drop_rate > 0:
                layers += [
                    nn.Conv2d(out_channels, out_channels, 
                              kernel_size=kernel_size, stride=stride, 
                              padding=padding, dilation=dilation, bias=False),
                    nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
                    nn.Dropout(drop_rate), 
                ] * (num_conv_layers - 1)
            else:
                layers += [
                    nn.Conv2d(out_channels, out_channels, 
                              kernel_size=kernel_size, stride=stride,
                              padding=padding, dilation=dilation, bias=False),
                    nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True), 
                ] * (num_conv_layers - 1)

        self.block = nn.Sequential(*layers)

    def forward(self, inputs):
        outputs = self.block(inputs)
        return outputs

###########################################################################

class UpconvBlock(nn.Module):
    r"""
    Decoder layer decodes the features along the expansive path.
    Args:
        in_channels (int) -- number of input features.
        out_channels (int) -- number of output features.
        upmode (str) -- Upsampling type. If "fixed" then a linear upsampling with scale factor
                        of two will be applied using bi-linear as interpolation method.
                        If deconv_1 is chosen then a non-overlapping transposed convolution will
                        be applied to upsample the feature maps. If deconv_1 is chosen then an
                        overlapping transposed convolution will be applied to upsample the feature maps.
    """

    def __init__(self, in_channels, out_channels, upmode="deconv_1"):
        super(UpconvBlock, self).__init__()

        if upmode == "fixed":
            layers = [nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True), ]
            layers += [nn.BatchNorm2d(in_channels),
                       nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False), ]

        elif upmode == "deconv_1":
            layers = [nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2, padding=0, dilation=1), ]

        elif upmode == "deconv_2":
            layers = [nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1, dilation=1), ]

        # Dense Upscaling Convolution
        elif upmode == "DUC":
            up_factor = 2
            upsample_dim = (up_factor ** 2) * out_channels
            layers = [nn.Conv2d(in_channels, upsample_dim, kernel_size=3, padding=1),
                      nn.BatchNorm2d(upsample_dim),
                      nn.ReLU(inplace=True),
                      nn.PixelShuffle(up_factor), ]

        else:
            raise ValueError("Provided upsampling mode is not recognized.")

        self.block = nn.Sequential(*layers)

    def forward(self, inputs):
        return self.block(inputs)


###########################################################################

class Unet(nn.Module):
    def __init__(self, n_classes, in_channels, filter_config=None, dropout_rate=0):
        super(Unet, self).__init__()

        self.in_channels = in_channels

        if not filter_config:
            filter_config = (64, 128, 256, 512, 1024, 2048)

        assert len(filter_config) == 6

        # Contraction Path
        self.encoder_1 = ConvBlock(self.in_channels, filter_config[0], num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 64x256x256
        self.encoder_2 = ConvBlock(filter_config[0], filter_config[1], num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 128x128x128
        self.encoder_3 = ConvBlock(filter_config[1], filter_config[2], num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 256x64x64
        self.encoder_4 = ConvBlock(filter_config[2], filter_config[3], num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 512x32x32
        self.encoder_5 = ConvBlock(filter_config[3], filter_config[4], num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 1024x16x16
        self.encoder_6 = ConvBlock(filter_config[4], filter_config[5], num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 2048x8x8
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Expansion Path
        self.decoder_1 = UpconvBlock(filter_config[5], filter_config[4], upmode="deconv_2")  # 1024x16x16
        self.conv1 = ConvBlock(filter_config[4] * 2, filter_config[4], num_conv_layers=2, drop_rate=dropout_rate)

        self.decoder_2 = UpconvBlock(filter_config[4], filter_config[3], upmode="deconv_2")  # 512x32x32
        self.conv2 = ConvBlock(filter_config[4], filter_config[3], num_conv_layers=2, drop_rate=dropout_rate)

        self.decoder_3 = UpconvBlock(filter_config[3], filter_config[2], upmode="deconv_2")  # 256x64x64
        self.conv3 = ConvBlock(filter_config[3], filter_config[2], num_conv_layers=2, drop_rate=dropout_rate)

        self.decoder_4 = UpconvBlock(filter_config[2], filter_config[1], upmode="deconv_2")  # 128x128x128
        self.conv4 = ConvBlock(filter_config[2], filter_config[1], num_conv_layers=2, drop_rate=dropout_rate)

        self.decoder_5 = UpconvBlock(filter_config[1], filter_config[0], upmode="deconv_2")  # 64x256x256
        self.conv5 = ConvBlock(filter_config[1], filter_config[0], num_conv_layers=2, drop_rate=dropout_rate)

        self.classifier = nn.Conv2d(filter_config[0], n_classes, kernel_size=1, stride=1, padding=0)  # classNumx256x256

    def forward(self, inputs):
        # set_trace()
        e1 = self.encoder_1(inputs)  # batch size x 64 x 256 x 256
        p1 = self.pool(e1)  # batch size x 64 x 128 x 128

        e2 = self.encoder_2(p1)  # batch size x 128 x 128 x 128
        p2 = self.pool(e2)  # batch size x 128 x 64 x 64

        e3 = self.encoder_3(p2)  # batch size x 256 x 64 x 64
        p3 = self.pool(e3)  # batch size x 256 x 32 x 32

        e4 = self.encoder_4(p3)  # batch size x 512 x 32 x 32
        p4 = self.pool(e4)  # batch size x 1024 x 16 x 16

        e5 = self.encoder_5(p4)  # batch size x 1024 x 16 x 16
        p5 = self.pool(e5)  # batch size x 1024 x 8 x 8

        e6 = self.encoder_6(p5)  # batch size x 2048 x 8 x 8

        d6 = self.decoder_1(e6)  # batch size x 1024 x 16 x 16

        
        skip1 = torch.cat((e5, d6), dim=1)  # batch size x 2048 x 16 x 16

        d6_proper = self.conv1(skip1)  # batch size x 1024 x 16 x 16

        d5 = self.decoder_2(d6_proper)  # batch size x 512 x 32 x 32

        skip2 = torch.cat((e4, d5), dim=1)  # batch size x 1024 x 32 x 32

        d5_proper = self.conv2(skip2)  # batch size x 512 x 32 x 32

        d4 = self.decoder_3(d5_proper)  # batch size x 256 x 64 x 64

        skip3 = torch.cat((e3, d4), dim=1)  # batch size x 512 x 64 x 64

        d4_proper = self.conv3(skip3)  # batch size x 256 x 64 x 64

        d3 = self.decoder_4(d4_proper)  # batch size x 128 x 128 x 128

        skip4 = torch.cat((e2, d3), dim=1)  # batch size x 256 x 128 x 128

        d3_proper = self.conv4(skip4)  # batch size x 128 x 128 x 128

        d2 = self.decoder_5(d3_proper)  # batch size x 64 x 256 x 256

        skip5 = torch.cat((e1, d2), dim=1)  # batch size x 128 x 256 x 256

        d2_proper = self.conv5(skip5)  # batch size x 64 x 256 x 256

        d1 = self.classifier(d2_proper)  # batch size x classNum x 256 x 256

        return d1

## **Customized loss function**

In [ ]:
#@title Loss function

class BinaryTverskyFocalLoss(nn.Module):
    r"""
    Pytorch versiono of tversky focal loss proposed in paper
    'A novel focal Tversky loss function and improved Attention U-Net for lesion segmentation'
    (https://arxiv.org/abs/1810.07842)
    Arguments:
        smooth (float): A float number to smooth loss, and avoid NaN error, default: 1
        alpha (float): Hyperparameters alpha, paired with (1 - alpha) to shift emphasis to improve recall
        gamma (float): Tversky index, default: 1.33
    Returns:
        Loss tensor
    """

    def __init__(self, smooth=1, alpha=0.7, gamma=1.33):
        super(BinaryTverskyFocalLoss, self).__init__()
        self.smooth = smooth
        self.alpha = alpha
        self.beta = 1 - self.alpha
        self.gamma = gamma

    def forward(self, predict, target):
        # shape: [N, C, *]
        assert predict.shape[0] == target.shape[0], "predict & target batch size do not match"

        # no reduction, same as original paper
        predict = predict.contiguous().view(-1)
        target = target.contiguous().view(-1)

        num = (predict * target).sum() + self.smooth
        den = (predict * target).sum() + self.alpha * ((1 - predict) * target).sum() \
              + self.beta * (predict * (1 - target)).sum() + self.smooth
        loss = torch.pow(1 - num / den, 1 / self.gamma)

        return loss


class TverskyFocalLoss(nn.Module):
    r"""
    Tversky focal loss
    Arguments:
        weight (torch.tensor): Weight array of shape [num_classes,]
        ignore_index (int): Class index to ignore
        predict (torch.tensor): Predicted tensor of shape [N, C, *]
        target (torch.tensor): Target tensor with the same shape as predict.
        other args pass to BinaryTverskyFocalLoss
    Returns:
        same as BinaryTverskyFocalLoss
    """

    def __init__(self, weight=None, ignore_index=-100, **kwargs):
        super(TverskyFocalLoss, self).__init__()
        self.kwargs = kwargs
        self.weight = weight
        self.ignore_index = ignore_index

    def forward(self, predict, target):
        nclass = predict.shape[1]
        if predict.shape == target.shape:
            pass
        elif len(predict.shape) == 4:
            target = F.one_hot(target, num_classes=nclass).permute(0, 3, 1, 2).contiguous()
        else:
            assert 'predict shape not applicable'

        tversky = BinaryTverskyFocalLoss(**self.kwargs)
        total_loss = 0
        weight = torch.Tensor([1. / nclass] * nclass).cuda() if self.weight is None else self.weight
        predict = F.softmax(predict, dim=1)

        for i in range(nclass):
            if i != self.ignore_index:
                tversky_loss = tversky(predict[:, i], target[:, i])
                assert weight.shape[0] == nclass, \
                    'Expect weight shape [{}], get[{}]'.format(nclass, weight.shape[0])
                tversky_loss *= weight[i]
                total_loss += tversky_loss

        return total_loss


class BalancedTverskyFocalLoss(nn.Module):
    r"""
    Tversky focal loss weighted by inverse of label frequency
    Arguments:
        ignore_index (int): Class index to ignore
        predict (torch.tensor): Predicted tensor of shape [N, C, *]
        target (torch.tensor): Target tensor either in shape [N,*] or of same shape with predict
        other args pass to BinaryTverskyFocalLoss
    Returns:
        same as TverskyFocalLoss
    """

    def __init__(self, ignore_index=-100, **kwargs):
        super(BalancedTverskyFocalLoss, self).__init__()
        self.kwargs = kwargs
        self.ignore_index = ignore_index

    def forward(self, predict, target):
        # get class weights
        unique, unique_counts = torch.unique(target, return_counts=True)
        # calculate weight for only valid indices
        unique_counts = unique_counts[unique != self.ignore_index]
        unique = unique[unique != self.ignore_index]
        ratio = unique_counts.float() / torch.numel(target)
        weight = (1. / ratio) / torch.sum(1. / ratio)

        lossWeight = torch.ones(predict.shape[1]).cuda() * 0.00001
        for i in range(len(unique)):
            lossWeight[unique[i]] = weight[i]

        # loss
        loss = TverskyFocalLoss(weight=lossWeight, ignore_index=self.ignore_index, **self.kwargs)

        return loss(predict, target)


class TverskyFocalCELoss(nn.Module):
    """
    Combination of tversky focal loss and cross entropy loss though summation
    Arguments:
        loss_weight (tensor): a manual rescaling weight given to each class. If given, has to be a Tensor of size C
        tversky_weight (float): Weight on tversky focal loss for the summation, while weight on cross entropy loss
                                is (1 - tversky_weight)
        tversky_smooth (float): A float number to smooth tversky focal loss, and avoid NaN error, default: 1
        tversky_alpha (float):
        tversky_gamma (float):
        ignore_index (int): Class index to ignore
    Returns:
        Loss tensor
    """

    def __init__(self, loss_weight=None, tversky_weight=0.5, tversky_smooth=1, tversky_alpha=0.7,
                 tversky_gamma=0.9, ignore_index=-100):
        super(TverskyFocalCELoss, self).__init__()
        self.loss_weight = loss_weight
        self.tversky_weight = tversky_weight
        self.tversky_smooth = tversky_smooth
        self.tversky_alpha = tversky_alpha
        self.tversky_gamma = tversky_gamma
        self.ignore_index = ignore_index

    def forward(self, predict, target):
        assert predict.shape[0] == target.shape[0], "predict & target batch size do not match"

        tversky = TverskyFocalLoss(weight=self.loss_weight, ignore_index=self.ignore_index, smooth=self.tversky_smooth,
                                   alpha=self.tversky_alpha, gamma=self.tversky_gamma)
        ce = nn.CrossEntropyLoss(weight=self.loss_weight, ignore_index=self.ignore_index)
        loss = self.tversky_weight * tversky(predict, target) + (1 - self.tversky_weight) * ce(predict, target)

        return loss


class BalancedTverskyFocalCELoss(nn.Module):
    r"""
    Combination of tversky focal loss and cross entropy loss weighted by inverse of label frequency
    Arguments:
        ignore_index (int): Class index to ignore
        predict (torch.tensor): Predicted tensor of shape [N, C, *]
        target (torch.tensor): Target tensor either in shape [N,*] or of same shape with predict
        other args pass to DiceCELoss, excluding loss_weight
    Returns:
        Same as TverskyFocalCELoss
    """

    def __init__(self, ignore_index=-100, **kwargs):
        super(BalancedTverskyFocalCELoss, self).__init__()
        self.ignore_index = ignore_index
        self.kwargs = kwargs

    def forward(self, predict, target):
        # get class weights
        unique, unique_counts = torch.unique(target, return_counts=True)
        # calculate weight for only valid indices
        unique_counts = unique_counts[unique != self.ignore_index]
        unique = unique[unique != self.ignore_index]
        ratio = unique_counts.float() / torch.numel(target)
        weight = (1. / ratio) / torch.sum(1. / ratio)

        lossWeight = torch.ones(predict.shape[1]).cuda() * 0.00001
        for i in range(len(unique)):
            lossWeight[unique[i]] = weight[i]

        loss = TverskyFocalCELoss(loss_weight=lossWeight, **self.kwargs)

        return loss(predict, target)

## **Model fitting (training + validation)**

In [ ]:
#@title Functions to train the model using the training set and do validation on the validation dataset.

def train(trainData, model, optimizer, criterion, device, train_loss=[]):
    """
        Train the model using provided training dataset.
        Params:
            trainData (DataLoader object) -- Batches of image chips from PyTorch custom dataset (AquacultureData).
            model -- Choice of segmentation model.
            optimizer -- Chosen optimization algorithm to update model parameters.
            criterion -- Chosen function to calculate loss over training samples.
            gpu (bool, optional) -- Decide whether to use GPU, default is True.
            train_loss (empty list, optional) -- ???????????????????????????
    """

    model.train()

    # Mini batch iteration
    train_epoch_loss = 0
    train_batches = len(trainData)

    for img_chips, labels in trainData:

        img = img_chips.to(device)
        label = labels.to(device)

        optimizer.zero_grad()

        pred = model(img)

        loss = eval(criterion)(pred, label)
        train_epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

    train_loss.append(train_epoch_loss / train_batches)
    print('Training loss: {:.4f}'.format(train_epoch_loss / train_batches))

##########################################################

def validate(valData, model, criterion, device, val_loss=[]):
    """
        Evaluate the model on separate Landsat scenes.
        Params:
            valData (DataLoader object) -- Batches of image chips from PyTorch custom dataset(AquacultureData)
            model -- Choice of segmentation Model.
            criterion -- Chosen function to calculate loss over validation samples.
            buffer: Buffer added to the targeted grid when creating dataset. This allows loss to calculate
                at non-buffered region.
            gpu (binary,optional): Decide whether to use GPU, default is True
            valLoss (empty list): To record average loss for each epoch
    """

    model.eval()

    # mini batch iteration
    eval_epoch_loss = 0

    for img_chips, labels in valData:

        img = Variable(img_chips, requires_grad=False)
        label = Variable(labels, requires_grad=False)

        img = img_chips.to(device)
        label = labels.to(device)

        pred = model(img)

        loss = eval(criterion)(pred, label)
        eval_epoch_loss += loss.item()

    print('validation loss: {}'.format(eval_epoch_loss / len(valData)))

    if val_loss != None:
        val_loss.append(float(eval_epoch_loss / len(valData)))

##########################################################

def epochIterater(trainData, valData, model, criterion, WorkingFolder, initial_lr, num_epochs):
    r"""
    Epoch iteration for train and evaluation.
    
    Arguments:
    trainData (dataloader object): Batch grouped data to train the model.
    evalData (dataloader object): Batch grouped data to evaluate the model.
    model (pytorch.nn.module object): initialized model.
    initial_lr(float): The initial learning rate.
    num_epochs (int): User-defined number of epochs to run the model.
    
    """

    train_loss = []
    val_loss = []
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    if device.type == "cuda":
        print('----------GPU available----------')
        model = model.to(device)
    else:
        print('----------No GPU available, using CPU instead----------')
        model = model
    
    writer = SummaryWriter(WorkingFolder)
    optimizer = optim.Adam(model.parameters(),
                           lr=initial_lr,
                           betas=(0.9, 0.999),
                           eps=1e-08,
                           weight_decay=5e-4,
                           amsgrad=False)
    
    scheduler = optim.lr_scheduler.StepLR(optimizer,
                                          step_size=3,
                                          gamma=0.90)
    
    for t in range(num_epochs):
        print("Epoch [{}/{}]".format(t + 1, num_epochs))
        start_epoch = datetime.now()

        train(trainData, model, optimizer, criterion, device, train_loss=train_loss)
        validate(valData, model, criterion, device, val_loss=val_loss)

        scheduler.step()
        print("LR: {}".format(scheduler.get_last_lr()))

        writer.add_scalars("Loss", 
                           {"train loss": train_loss[t],
                            "validation loss": val_loss[t]},
                           t + 1)
    
    writer.close()

    duration_in_sec = (datetime.now() - start_epoch).seconds
    duration_format = str(timedelta(seconds=duration_in_sec))
    print("--------------- Training finished in {} ---------------".format(duration_format))

## **Evaluation and accuracy metrics**

In [ ]:
#@title Metrics and evaluation functions


class Evaluator(object):
    def __init__(self, num_class):
        self.num_class = num_class
        self.confusion_matrix = np.zeros((self.num_class,)*2)

    def Pixel_Accuracy(self):
        Acc = np.diag(self.confusion_matrix).sum() / self.confusion_matrix.sum()
        return Acc

    def Pixel_Accuracy_Class(self):
        Acc = np.diag(self.confusion_matrix) / self.confusion_matrix.sum(axis=1)
        Acc = np.nanmean(Acc)
        return Acc

    def Mean_Intersection_over_Union(self):
        MIoU = np.diag(self.confusion_matrix) / (
                    np.sum(self.confusion_matrix, axis=1) + np.sum(self.confusion_matrix, axis=0) -
                    np.diag(self.confusion_matrix))
        MIoU = np.nanmean(MIoU)
        return MIoU

    def Frequency_Weighted_Intersection_over_Union(self):
        freq = np.sum(self.confusion_matrix, axis=1) / np.sum(self.confusion_matrix)
        iu = np.diag(self.confusion_matrix) / (
                    np.sum(self.confusion_matrix, axis=1) + np.sum(self.confusion_matrix, axis=0) -
                    np.diag(self.confusion_matrix))

        FWIoU = (freq[freq > 0] * iu[freq > 0]).sum()
        return FWIoU

    def _generate_matrix(self, gt_image, pre_image):
        mask = (gt_image >= 0) & (gt_image < self.num_class)
        label = self.num_class * gt_image[mask].astype('int') + pre_image[mask]
        count = np.bincount(label, minlength=self.num_class**2)
        confusion_matrix = count.reshape(self.num_class, self.num_class)
        return confusion_matrix

    def add_batch(self, gt_image, pre_image):
        assert gt_image.shape == pre_image.shape
        self.confusion_matrix += self._generate_matrix(gt_image, pre_image)

    def reset(self):
        self.confusion_matrix = np.zeros((self.num_class,) * 2)

#########################

def do_accuracy_evaluation(model, dataloader, num_classes):
    evaluator = Evaluator(num_classes)

    model.eval()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = torch.max(outputs.data, 1)

            # add batch to evaluator
            evaluator.add_batch(labels.cpu().numpy(), preds.cpu().numpy())

    # calculate evaluation metrics
    pixel_accuracy = evaluator.Pixel_Accuracy()
    mean_accuracy = evaluator.Pixel_Accuracy_Class()
    mean_IoU = evaluator.Mean_Intersection_over_Union()
    frequency_weighted_IoU = evaluator.Frequency_Weighted_Intersection_over_Union()

    return pixel_accuracy, mean_accuracy, mean_IoU, frequency_weighted_IoU

## **Prediction**

In [ ]:
#@title Complete the code for performing prediction


def do_prediction(testData, model, overlap, device, save_dir):
    r"""
    Use train model to predict on unseen data.
    Arguments:
            testData (custom iterator) -- Batches of image chips from PyTorch 
                                          custom dataset.
            model (ordered Dict) -- trained model.
            overlap (int) -- amount of overlap between prediction chips.
            device (str) -- Either "cpu" or "cuda".
            save_dir (str) -- Directory to save the prediction output.
    """
    
    # Create directories to save the predicted output
    save_dir_hard = Path(save_dir) / "HardScore"
    save_dir_soft = Path(save_dir) / "SoftScore"

    os.makedirs(save_dir_hard, exist_ok=True)
    os.makedirs(save_dir_soft, exist_ok=True)

    # Start inference on test data
    print("--------------------- Start Inference(Test) ---------------------")
    start = datetime.now()

    # Get the test data, metadata, and tile information
    # add your code here
    testData, meta, tile = testData

    # Define the output file names and metadata for the hard and soft scores
    name_prob = "prob_c{}_r{}".format(tile[0], tile[1])
    name_crisp = "crisp_c{}_r{}.rst".format(tile[0], tile[1])

    meta_hard = meta.copy()
    meta_hard.update({
        "dtype": "uint8",
        "count": 1,
    })

    meta_soft = meta.copy()
    meta_soft.update({
        "dtype": "float32",
        "count": 1,
    })

    model = model.to(device)

    ##### Add your code to put the model in evaluation mode. (1 line)
    model.eval()

    
    canvas_score_ls = []
    
    ##### Create a canvas (call it "h_canvas") with the same height, width and datatype 
    ##### from "meta_hard" to hold the score values and initialize it to zeros.
    ##### Add your code here. (Expected 1 line)
    h_canvas = np.zeros(shape=(meta["width"], meta["height"]), dtype=meta_hard["dtype"])


    # Loop over batches of image chips and indices.
    for img_chips, index_batch in testData:

        img = Variable(img_chips, requires_grad=False)
        img = img_chips.to(device) # size: B X in_channels X W X H

        ##### Forward pass through the model to get the predictions and assign it to a variable called "pred".
        ##### Add your code here. (Expected 1 line)
        pred = model.forward(img)
        
        ##### Normalize the model output using "softmax" And assign it to a 
        ##### variable called "pred_prob".
        ##### Add your code here (Expected 1 line)
        pred_prob = nn.Softmax(pred)
        #pred_prob = F.softmax(pred, dim=1)

        # Get the dimensions of the prediction
        batch, n_class, height, width = pred_prob.size()

        # Calculate the score width and score height based on the overlap parameter
        score_width = (width // 2) - overlap
        score_height = (height // 2) - overlap

        # Loop over the batch and assign the predicted scores to the canvas
        for i in range(batch):

            # creating a new tuple index containing the coordinates which makes it 
            # easier to index into the "h_canvas" and arrays in the "canvas_score_ls" later on in the code.
            index = (index_batch[0][i], index_batch[1][i])

            # Get the hard scores by taking the argmax of the prediction
            prediction_hard = pred_prob.max(dim=1)[1][:, overlap:-overlap, overlap:-overlap].cpu().numpy()[i, :, :]
            
            # add the batch dimension to the "prediction_hard" array and convert its data types.
            prediction_hard = np.expand_dims(prediction_hard, axis=0).astype(meta_hard["dtype"])

            # The "prediction_hard" values are assigned to a slice of "h_canvas", 
            # effectively updating the pixels in the original image corresponding
            # to the current image chip in the batch with the predicted values for that chip.
            ##### Add your code here. (Expected 1 line)
            h_canvas = h_canvas.insert(prediction_hard)
            #h_canvas[:, index[0] - score_width : index[0] + score_width, index[1] - score_height : index[1] + score_height] = prediction_hard


            for n in range(1, n_class):
                # Extract probability map for class n from predicted probabilities tensor
                prediction_soft = pred_prob[:, n, :, :].data[i][overlap:-overlap, overlap:-overlap].cpu().numpy() * 100
                # Add an extra dimension to the probability map to match the expected shape
                prediction_soft = np.expand_dims(prediction_soft, axis=0).astype(meta_soft["dtype"])

                try:
                    # Update existing canvas for class n with the new probability map
                    canvas_score_ls[n][:, index[0] - score_width : index[0] + score_width,
                    index[1] - score_height: index[1] + score_height] = prediction_soft
                except:
                    # Create a new canvas for class n and initialize it with zeros
                    canvas_score_single = np.zeros((1, meta_soft['height'],
                                                    meta_soft['width']), dtype=meta_soft['dtype'])

                    # Update the new canvas with the new probability map slice by slice
                    canvas_score_single[:, index[0] - score_width: index[0] + score_width,
                    index[1] - score_height: index[1] + score_height] = prediction_soft
                    
                    # Add the new canvas to the list of canvases for all classes
                    canvas_score_ls.append(canvas_score_single)
    
    # write the hard classification results to an output raster.
    ##### Use "save_dir_hard", "name_crisp" and "meta_hard".
    ##### Add your code here. (Expected 2 line)
    with rasterio.open(fp=os.path.join(save_dir_hard, name_crisp), mode='w', **meta_hard) as new_dataset:
      new_dataset.write(h_canvas, 1)

    
    # loop through each class (excluding the background class) and creates a 
    # new raster file for each class.
    ##### Add your code here. (Expected 4 line)
    ##### hint: use this code to get a proper name for the prediction output 
    ##### for each class: name_prob_updated = f"{name_prob}_Cat_{n}.tif" 

    for n in range(1, n_classes):
      name_prob_updated = f"{name_prob}_Cat_{n}.tif"
      with rasterio.open(fp=os.path.join(save_dir_hard, name_prob_updated), mode='w', **meta_hard) as dataset_class:
        dataset_class.write(np.zeros(shape=(meta["width"], meta["height"]), dtype=meta_hard["dtype"]))
        #dst.write(canvas_score_ls[n])


    
    duration_in_sec = (datetime.now() - start).seconds
    duration_format = str(timedelta(seconds=duration_in_sec))
    print("---------------- Inference finished in {} seconds ----------------".format(duration_format))

## **Running through the pipeline**

#### **Steps to train the model from scratch**

In [ ]:
src_dir = "/content/gdrive/MyDrive/GEOG315/A5_resources"
dataset_name = "Global"
transform = ["hflip", "vflip", "rotate"]

n_classes = 2
in_channels = 6
filter_config = (32, 64, 128, 256, 512, 1024)
dropout_rate = 0.1

criterion = "BalancedTverskyFocalCELoss()"
WorkingFolder = "/content/gdrive/MyDrive/GEOG315/test_a5"
initial_lr = 0.15
epochs = 15 

In [ ]:
# create a 'train_dataset' object using the 'AquacultureData' class.
train_dataset = AquacultureData(src_dir, 
                                usage="train", 
                                dataset_name=dataset_name, 
                                apply_normalization=False, 
                                transform=transform)

100%|██████████| 1188/1188 [01:34<00:00, 12.57it/s]

--------------1188 patches cropped--------------


In [ ]:
# create a Pytorch data loader called 'train_loader' that loads data from the 
# 'train_dataset' dataset, split it into batches, convert is to tensor and move 
# the data to GPU if available.
 
train_loader = DataLoader(train_dataset,
                          batch_size = 16, 
                          shuffle = True)

In [ ]:
# create a 'validation_dataset' object using the 'AquacultureData' class.
validation_dataset = AquacultureData(src_dir, 
                                     usage="validation", 
                                     dataset_name=dataset_name, 
                                     apply_normalization=False)

100%|██████████| 239/239 [00:07<00:00, 31.69it/s]

--------------239 patches cropped--------------


In [ ]:
# create a Pytorch data loader for the 'validation_dataset'.
val_loader = DataLoader(validation_dataset, 
                        batch_size = 1, 
                        shuffle = False)

In [ ]:
# initialize the model
model = Unet(n_classes, 
             in_channels, 
             filter_config, 
             dropout_rate)

In [ ]:
# model fitting
epochIterater(train_loader, 
              val_loader, 
              model, 
              criterion, 
              WorkingFolder, 
              initial_lr, 
              epochs)

----------GPU available----------
Epoch [1/15]
Training loss: 0.4280
validation loss: 0.5395882622218032
LR: [0.15]
Epoch [2/15]
Training loss: 0.3430
validation loss: 0.43622303601338774
LR: [0.15]
Epoch [3/15]
Training loss: 0.3482
validation loss: 2.023297564255144
LR: [0.135]
Epoch [4/15]
Training loss: 0.3443
validation loss: 119.25499013078762
LR: [0.135]
Epoch [5/15]
Training loss: 0.3521
validation loss: 0.42033863903089547
LR: [0.135]
Epoch [6/15]
Training loss: 0.3431
validation loss: 1.8150634109724515
LR: [0.12150000000000001]
Epoch [7/15]
Training loss: 0.3381
validation loss: 0.6393037419688252
LR: [0.12150000000000001]
Epoch [8/15]
Training loss: 0.3416
validation loss: 0.44686710323249945
LR: [0.12150000000000001]
Epoch [9/15]
Training loss: 0.3378
validation loss: 89.27381250549061
LR: [0.10935000000000002]
Epoch [10/15]
Training loss: 0.3398
validation loss: 0.5627945718904919
LR: [0.10935000000000002]
Epoch [11/15]
Training loss: 0.3396
validation loss: 0.61484984193

In [ ]:
# save the trained model parameters.
torch.save(model.state_dict(), 
           os.path.join(Path(WorkingFolder), "trained_unet_final_state.pth"))

In [ ]:
test = do_accuracy_evaluation(model.cuda(), val_loader, 2)

#### **Fine-tuning the model on a a new dataset**

In [ ]:
src_dir = "/content/gdrive/MyDrive/GEOG315/A5_resources" # change if neeeded
dataset_name = "Fine_tune_dataset"
transform = ["hflip", "vflip", "rotate"]
params_dir = "/content/gdrive/MyDrive/GEOG315/test_a5/trained_unet_final_state.pth"

n_classes = 2
in_channels = 6
filter_config = (32, 64, 128, 256, 512, 1024)
dropout_rate = 0.15

criterion = "BalancedTverskyFocalCELoss()"
WorkingFolder = "/content/gdrive/MyDrive/GEOG315/test_a5"
initial_lr = 0.01
epochs = 50

In [ ]:
train_dataset = AquacultureData(src_dir, 
                                usage="train", 
                                dataset_name=dataset_name, 
                                apply_normalization=False, 
                                transform=transform)

100%|██████████| 758/758 [00:39<00:00, 19.16it/s]

--------------758 patches cropped--------------


In [ ]:
train_loader = DataLoader(train_dataset,
                          batch_size = 16, 
                          shuffle = True)

In [ ]:
validation_dataset = AquacultureData(src_dir, 
                                     usage="validation", 
                                     dataset_name=dataset_name, 
                                     apply_normalization=False)

100%|██████████| 122/122 [00:03<00:00, 38.24it/s]

--------------122 patches cropped--------------


In [ ]:
val_loader = DataLoader(validation_dataset, 
                        batch_size = 1, 
                        shuffle = False)

In [ ]:
model = Unet(n_classes, 
             in_channels, 
             filter_config, 
             dropout_rate)

In [ ]:
params_dir

'/content/gdrive/MyDrive/GEOG315/test_a5/trained_unet_final_state.pth'

In [ ]:
# Load trained model parameters
model = load_params(params_dir=params_dir, 
                    model=model, 
                    freeze_params=[0])

#### **Doing prediction (Inference)**

In [ ]:
src_dir = "/content/gdrive/MyDrive/GEOG315/A5_resources/prediction_scenes"
csv_name = "pond_scenes_inference.csv"
patch_size = 256
overlap = 28

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

In [ ]:
# If you are running the prediction at a different time with a new session
# then you need to initialize the model and load the parameters. 
# OTHERWISE IGNORE THIS CELL

params_dir = "/content/gdrive/MyDrive/GEOG315/A5_resources/trained_model_parameters/trained_unet_final_state.pth"

model = Unet(n_classes, in_channels, filter_config, dropout_rate)
model = load_params(params_dir, model, freeze_params=None)

In [ ]:
def load_data_pred(usage, csv_name, patch_size, overlap, catalog_row):
    pred_dataset = AquacultureData(src_dir,
                                   usage = usage,
                                   apply_normalization=False,
                                   csv_name = csv_name,
                                   patch_size = patch_size,
                                   overlap = overlap,
                                   catalog_index=catalog_row)
    
    data_loader = DataLoader(pred_dataset, batch_size=1, shuffle=False)
    meta = pred_dataset.meta
    tile = pred_dataset.tile
    
    return data_loader, meta, tile

In [ ]:
tile_count = len(pd.read_csv(os.path.join(src_dir, csv_name)))

In [ ]:
for i in range(tile_count):
    pred_data = load_data_pred("inference", csv_name, patch_size, overlap, i)
    do_prediction(pred_data, model, overlap, device, save_dir)

RasterioIOError: ignored

**<font color='red'> 17/25 points on CA2</font>**

**<font color='red'> check lines 75, 102, 143 that I added.</font>**